In [1]:
import numpy as np
import datasets 
from datasets import load_dataset
from accelerate import Accelerator
import torch
from torch.utils.data.dataloader import DataLoader
import transformers
from transformers import (
    AutoTokenizer,
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    default_data_collator,
    get_scheduler,
    set_seed,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = load_dataset("json", data_files="train.json")
dataset_test = load_dataset("json", data_files="test.json")
dataset_eval = load_dataset("json", data_files="eval.json")
dataset['eval'] = dataset_eval['train']
dataset['test'] = dataset_test['train']
dataset
mt5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

Found cached dataset json (C:/Users/KevinChou/.cache/huggingface/datasets/json/default-c28acd79c90df9df/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (C:/Users/KevinChou/.cache/huggingface/datasets/json/default-c1d957cf7d7b7168/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (C:/Users/KevinChou/.cache/huggingface/datasets/json/default-656e565e73707297/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

d:\AI_project\AI_env\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
def tokenize_sample_data(data):
  # Max token size is 14536 and 215 for inputs and labels, respectively.
  # Here I restrict these token size.
  input_feature = mt5_tokenizer(data["article"], truncation=True, max_length=256)
  label = mt5_tokenizer(data["title"], truncation=True, max_length=64)
  return {
    "input_ids": input_feature["input_ids"],
    "attention_mask": input_feature["attention_mask"],
    "labels": label["input_ids"],
  }

tokenized_ds = dataset.map(
  tokenize_sample_data,
  remove_columns=["id", "title", "url", "article"],
  batched=True,
  batch_size=128)

tokenized_ds

Loading cached processed dataset at C:\Users\KevinChou\.cache\huggingface\datasets\json\default-c28acd79c90df9df\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-d2e7ffa472b639b7.arrow


Map:   0%|          | 0/3777 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\KevinChou\.cache\huggingface\datasets\json\default-c1d957cf7d7b7168\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-47b297e28f804521.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 30221
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3777
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3777
    })
})

In [4]:
# see https://huggingface.co/docs/transformers/main_classes/configuration
mt5_config = AutoConfig.from_pretrained(
  "google/mt5-small",
  max_length=128,
  length_penalty=0.6,
  no_repeat_ngram_size=2,
  num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-small", config=mt5_config)
         .to(device))

In [5]:

data_collator = DataCollatorForSeq2Seq(
  mt5_tokenizer,
  model=model,
  return_tensors="pt")

In [6]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = mt5_tokenizer(arg)
  return mt5_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, mt5_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = mt5_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = mt5_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
  text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
  sent_tokenizer_tw = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_tw.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_tw.tokenize(l))) for l in text_labels]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [7]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

metrics_func([preds, labels])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
d:\AI_project\AI_env\lib\site-packages\transformers\generation\utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


{'rouge1': 0.1353822393822394,
 'rouge2': 0.06066053511705686,
 'rougeL': 0.13491165114694526,
 'rougeLsum': 0.1339625344195403}

In [8]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
  output_dir = "mt5-summarize-tw",
  log_level = "error",
  num_train_epochs = 10,
  learning_rate = 5e-4,
  lr_scheduler_type = "linear",
  warmup_steps = 90,
  optim = "adafactor",
  weight_decay = 0.01,
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 1,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 100,
  predict_with_generate=True,
  generation_max_length = 128,
  save_steps = 500,
  logging_steps = 10,
  push_to_hub = False
)

In [9]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  compute_metrics = metrics_func,
  train_dataset = tokenized_ds["train"],
  eval_dataset = tokenized_ds["eval"].select(range(20)),
  tokenizer = mt5_tokenizer,
)

trainer.train()

  0%|          | 0/9440 [00:00<?, ?it/s]

{'loss': 22.0758, 'learning_rate': 5.555555555555555e-05, 'epoch': 0.01}
{'loss': 17.5624, 'learning_rate': 0.0001111111111111111, 'epoch': 0.02}
{'loss': 11.9687, 'learning_rate': 0.00016666666666666666, 'epoch': 0.03}
{'loss': 8.6245, 'learning_rate': 0.0002222222222222222, 'epoch': 0.04}
{'loss': 7.0345, 'learning_rate': 0.0002777777777777778, 'epoch': 0.05}
{'loss': 6.1489, 'learning_rate': 0.0003333333333333333, 'epoch': 0.06}
{'loss': 5.7781, 'learning_rate': 0.0003888888888888889, 'epoch': 0.07}
{'loss': 5.5156, 'learning_rate': 0.0004444444444444444, 'epoch': 0.08}
{'loss': 5.1996, 'learning_rate': 0.0005, 'epoch': 0.1}
{'loss': 4.9822, 'learning_rate': 0.0004994652406417112, 'epoch': 0.11}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 4.09685754776001, 'eval_rouge1': 0.3410369928508299, 'eval_rouge2': 0.16781601279675154, 'eval_rougeL': 0.31374166261696534, 'eval_rougeLsum': 0.31380862534484477, 'eval_runtime': 8.8043, 'eval_samples_per_second': 2.272, 'eval_steps_per_second': 2.272, 'epoch': 0.11}
{'loss': 5.07, 'learning_rate': 0.0004989304812834225, 'epoch': 0.12}
{'loss': 4.9996, 'learning_rate': 0.0004983957219251337, 'epoch': 0.13}
{'loss': 4.7466, 'learning_rate': 0.000497860962566845, 'epoch': 0.14}
{'loss': 4.7188, 'learning_rate': 0.0004973262032085562, 'epoch': 0.15}
{'loss': 4.5726, 'learning_rate': 0.0004967914438502674, 'epoch': 0.16}
{'loss': 4.487, 'learning_rate': 0.0004962566844919787, 'epoch': 0.17}
{'loss': 4.539, 'learning_rate': 0.0004957219251336899, 'epoch': 0.18}
{'loss': 4.4689, 'learning_rate': 0.0004951871657754011, 'epoch': 0.19}
{'loss': 4.4616, 'learning_rate': 0.0004946524064171123, 'epoch': 0.2}
{'loss': 4.4107, 'learning_rate': 0.0004941176470588236, 'epoch': 0.21}


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 3.0270307064056396, 'eval_rouge1': 0.49498861979167535, 'eval_rouge2': 0.2767119096578238, 'eval_rougeL': 0.4009384462384352, 'eval_rougeLsum': 0.4253422842148876, 'eval_runtime': 11.2296, 'eval_samples_per_second': 1.781, 'eval_steps_per_second': 1.781, 'epoch': 9.85}
{'loss': 2.6675, 'learning_rate': 6.951871657754011e-06, 'epoch': 9.86}
{'loss': 2.5515, 'learning_rate': 6.417112299465241e-06, 'epoch': 9.87}
{'loss': 2.5801, 'learning_rate': 5.882352941176471e-06, 'epoch': 9.88}
{'loss': 2.523, 'learning_rate': 5.3475935828877e-06, 'epoch': 9.89}
{'loss': 2.541, 'learning_rate': 4.812834224598931e-06, 'epoch': 9.9}
{'loss': 2.6464, 'learning_rate': 4.2780748663101604e-06, 'epoch': 9.91}
{'loss': 2.578, 'learning_rate': 3.7433155080213903e-06, 'epoch': 9.92}
{'loss': 2.6819, 'learning_rate': 3.2085561497326205e-06, 'epoch': 9.93}
{'loss': 2.5799, 'learning_rate': 2.67379679144385e-06, 'epoch': 9.94}
{'loss': 2.5014, 'learning_rate': 2.1390374331550802e-06, 'epoch': 9.95}

  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 3.029432773590088, 'eval_rouge1': 0.4995627369778739, 'eval_rouge2': 0.28425467934355125, 'eval_rougeL': 0.4104432806999512, 'eval_rougeLsum': 0.4347540743844628, 'eval_runtime': 11.1126, 'eval_samples_per_second': 1.8, 'eval_steps_per_second': 1.8, 'epoch': 9.95}
{'loss': 2.5658, 'learning_rate': 1.6042780748663103e-06, 'epoch': 9.96}
{'loss': 2.5938, 'learning_rate': 1.0695187165775401e-06, 'epoch': 9.97}
{'loss': 2.5668, 'learning_rate': 5.347593582887701e-07, 'epoch': 9.98}
{'loss': 2.5172, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 14888.1299, 'train_samples_per_second': 20.299, 'train_steps_per_second': 0.634, 'train_loss': 3.0734061154268555, 'epoch': 10.0}


TrainOutput(global_step=9440, training_loss=3.0734061154268555, metrics={'train_runtime': 14888.1299, 'train_samples_per_second': 20.299, 'train_steps_per_second': 0.634, 'train_loss': 3.0734061154268555, 'epoch': 10.0})

In [10]:
import os
from transformers import AutoModelForSeq2SeqLM

# save fine-tuned model in local
os.makedirs("./trained_for_summarization_tw", exist_ok=True)
if hasattr(trainer.model, "module"):
  trainer.model.module.save_pretrained("./trained_for_summarization_tw")
else:
  trainer.model.save_pretrained("./trained_for_summarization_tw")

# load local model
model = (AutoModelForSeq2SeqLM
         .from_pretrained("./trained_for_summarization_tw")
         .to(device))

In [14]:
from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

# Replace -100 (see above)
labels = np.where(labels != -100, labels, mt5_tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = mt5_tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = mt5_tokenizer.batch_decode(labels, skip_special_tokens=True)

# Show result
print("***** Input's Text *****")
print(dataset["test"]["article"][4])
print("***** Summary Text (True Value) *****")
print(text_labels[4])
print("***** Summary Text (Generated Text) *****")
print(text_preds[4])

***** Input's Text *****
***** Summary Text (True Value) *****
身材不是最重要!男性票選理想女友特質Top 10 第一名讓人猜不到
***** Summary Text (Generated Text) *****
「理想女友」外在條件佔比不高!第一名讓人覺得厭煩
